<h1 dir="rtl" style="font-family: Vazir; width: 85%;">تمرین شماره ۱: درس بازیابی هوشمند اطلاعات - دانشگاه تهران، پائیز ۱۴۰۳</h1>

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 18px;">
نام:
<br/>
شماره دانشجویی:

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;">
سوالات خودتان را می‌توانید از طریق ایمیل
<code>miladmohammadi@ut.ac.ir</code>
 از طراح تمرین ? بپرسید.

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 18px; color: red; font-weight: bold;">
قوانین و توضیحاتی آخر فایل تمرین حتما به دقت مطالعه شود.
</div>

In [ ]:
!pip install nltk
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn

In [1]:
import document_processing as dp
import inverted_index as ii
import query_processing as qp
import evaluation as ev

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


In [2]:
# Load the autoreload extension
%load_ext autoreload

# Set autoreload to Mode 1: Reload modules only when explicitly imported
%autoreload 1

# Mark specific modules for autoreloading
%aimport retrieval_models


import retrieval_models as rm

In [3]:
# Ensure NLTK data is downloaded
import nltk
import numpy as np
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


False

# **سوال اول:**

## Loading the dataset 1

In [4]:
# Paths to data files
corpus_file = 'data/LA.xml' #LA.xml
query_file = 'data/topics.xml' #topics.xml
relevance_file = 'data/jud-la.txt' #jud-la.txt
stopwords_file = 'data/stops_en.txt' #stops_en.txt

In [5]:
# 1. Data Loading and Preprocessing
dataset_type = 'type2'
# Load stopwords
stop_words = dp.load_stopwords(stopwords_file)

print("Loading and preprocessing documents...")
documents = dp.load_documents(corpus_file, dataset_type=dataset_type)
for idx, doc in enumerate(documents):
    doc.preprocess(stop_words=stop_words)
    if (idx + 1) % 100 == 0 or idx == len(documents) - 1:
        print(f"Processed {idx+1}/{len(documents)} documents")

print("Building inverted index...")
inverted_index = ii.InvertedIndex()
inverted_index.build_index(documents)

print("Loading and preprocessing queries...")
queries = qp.load_queries(query_file, dataset_type=dataset_type)
for query in queries:
    query.preprocess(stop_words=stop_words)

print("Loading relevance judgments...")
relevance_judgments = ev.load_relevance_judgments(relevance_file)

Loading and preprocessing documents...
Processed 100/113005 documents
Processed 200/113005 documents
Processed 300/113005 documents
Processed 400/113005 documents
Processed 500/113005 documents
Processed 600/113005 documents
Processed 700/113005 documents
Processed 800/113005 documents
Processed 900/113005 documents
Processed 1000/113005 documents
Processed 1100/113005 documents
Processed 1200/113005 documents
Processed 1300/113005 documents
Processed 1400/113005 documents
Processed 1500/113005 documents
Processed 1600/113005 documents
Processed 1700/113005 documents
Processed 1800/113005 documents
Processed 1900/113005 documents
Processed 2000/113005 documents
Processed 2100/113005 documents
Processed 2200/113005 documents
Processed 2300/113005 documents
Processed 2400/113005 documents
Processed 2500/113005 documents
Processed 2600/113005 documents
Processed 2700/113005 documents
Processed 2800/113005 documents
Processed 2900/113005 documents
Processed 3000/113005 documents
Processed 

## Example

In [6]:
print("\nSample run with BM25 (k=1.4, b=0.7)")
params = {'k': 1.4, 'b': 0.7}
evaluation_results = ev.evaluate_scoring_method(inverted_index,rm.bm25_score, queries, relevance_judgments, params=params)
print(evaluation_results)
print(f"MAP for BM25 (k={1.4}, b={0.7}): {evaluation_results['MAP']}")


Sample run with BM25 (k=1.4, b=0.7)
{'MAP': 0.3755, 'Precision@5': 0.422, 'Recall@10': 0.3157, 'nDCG@5': 0.5117}
MAP for BM25 (k=1.4, b=0.7): 0.3755


## Pseudo Relevance Feedback using Mixture Model

In [7]:
from tqdm import tqdm

print("\nPerforming Pseudo Relevance Feedback using Mixture Model")

# Parameters for PRF
top_n = 10  # Number of top documents to consider as pseudo-relevant
num_expansion_terms = 10  # Number of terms to select for query expansion
max_iterations = 10000
convergence_threshold = 1e-7
lambda_init = 0.5

# Import copy module for deep copying queries
import copy

# Create a list to store expanded queries
prf_queries = []

# Perform PRF for each query with progress bar
for query in tqdm(queries, desc="Processing Queries", unit="query"):
    # Make a deep copy of the query to avoid modifying the original
    prf_query = copy.deepcopy(query)
    
    # Perform Mixture Model PRF
    rm.mixture_model_prf(prf_query, inverted_index,
                         top_n=top_n,
                         num_expansion_terms=num_expansion_terms,
                         max_iterations=max_iterations,
                         convergence_threshold=convergence_threshold,
                         lambda_init=lambda_init)
    prf_queries.append(prf_query)

print("PRF completed for all queries.")

# Evaluate the expanded queries
print("\nEvaluating the expanded queries with BM25")
evaluation_results_prf = ev.evaluate_scoring_method(inverted_index, rm.bm25_score, prf_queries, relevance_judgments, params=params)
print(evaluation_results_prf)
print(f"MAP for PRF with Mixture Model: {evaluation_results_prf['MAP']}")


Performing Pseudo Relevance Feedback using Mixture Model


Processing Queries: 100%|██████████| 100/100 [00:05<00:00, 19.46query/s]


PRF completed for all queries.

Evaluating the expanded queries with BM25
{'MAP': 0.3808, 'Precision@5': 0.436, 'Recall@10': 0.3226, 'nDCG@5': 0.4964}
MAP for PRF with Mixture Model: 0.3808


# **نکات مهم**

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;">
    <p><strong>مهلت تحویل بدون جریمه:</strong> ۱۵ مهر ۱۴۰۲</p>
    <p><strong>مهلت تحویل با تاخیر (با جریمه):</strong> ۲۲ مهر ۱۴۰۲</p>
</div>
<h4 dir="rtl" style="font-family: Vazir; width: 85%;">فایل ارسالی شما باید با فرمت زیر نامگذاری شود: <code>IIR_CA1_STUDENTID.ipynb</code></h4>

<h4 dir="rtl" style="font-family: Vazir; width: 85%;">نحوه انجام این تمرین:</h4>
<ul dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;"> <li>برخی سوالات نیاز به نوشتن کد پایتون و محاسبه نتایج دارند کدها بایستی طبق فایل تمرین به طور کامل نوشته شوند.</li> <li> کدها و تفسیرهای هربخش را به طور مشخص در همین نوت‌بوک بنویسید. سعی کنید هربخش به طور مشخصی جداشده باشد و ساختار نوت‌بوک خوانا باشد.</li>  </ul>

<h4 dir="rtl" style="font-family: Vazir; width: 85%;">صداقت علمی:</h4> <ul dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;"> <li>ما نوت‌بوک‌های تعداد مشخصی از دانشجویان که به صورت تصادفی انتخاب می‌شوند، بررسی خواهیم کرد. این بررسی‌ها اطمینان حاصل می‌کنند که کدی که نوشتید واقعاً پاسخ‌های موجود در نوت‌بوک شما را تولید می‌کند. اگر پاسخ‌های صحیح را در نوت‌بوک خود بدون کدی که واقعاً آن پاسخ‌ها را تولید کند تحویل دهید، این یک مورد جدی از عدم صداقت علمی محسوب می‌شود.</li> <li>ما همچنین بررسی‌های خودکاری را برای تشخیص سرقت علمی در نوت‌بوک‌های کولب انجام خواهیم داد. کپی کردن کد از دیگران نیز یک مورد جدی از عدم صداقت علمی محسوب می‌شود.</li> </ul>

<h4 dir="rtl" style="font-family: Vazir; width: 85%;">توضیحات تکمیلی:</h4> <ul dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;">
<li>
خوانایی و دقت بررسی‌ها در گزارش نهایی از اهمیت ویژه‌ای برخوردار است. به تمرین‌هایی که به صورت کاغذی تحویل داده شوند یا به صورت عکس در سایت بارگذاری شوند، ترتیب اثری داده نخواهد شد.</li>
<li>
 همه‌ی کدهای پیوست گزارش بایستی قابلیت اجرای مجدد داشته باشند. در صورتی که برای اجرا مجدد آن‌ها نیاز به تنظیمات خاصی می‌باشد، بایستی تنظیمات مورد نیاز را نیز در گزارش خود ذکر کنید.  دقت کنید که  تمامی کدها باید توسط شما اجرا شده باشند و نتایج اجرا در فایل کدهای ارسالی مشخص باشد. به کدهایی که نتایج اجرای آن‌ها در فایل ارسالی مشخص نباشد نمره‌ای تعلق نمی‌گیرد.
</li>
<li>
تمرین تا یک هفته بعد از مهلت تعیین شده با تاخیر تحویل گرفته می‌شود. دقت کنید که شما جمعاً برای تمام تکالیف، ۱۴ روز زمان تحویل بدون جریمه دارید که تنها از ۷ روز آن برای هر تمرین می‌توانید استفاده کنید. در صورتی که این ۱۴ روز به اتمام رسیده باشد، به ازای هر روز تاخیر ده درصد جریمه می‌شود.
</li>
<li>توجه کنید این تمرین باید به صورت تک‌نفره انجام شود و پاسخ‌های ارائه شده باید نتیجه فعالیت فرد نویسنده باشد (همفکری و به اتفاق هم نوشتن تمرین نیز ممنوع است). در صورت مشاهده
 تشابه به همه افراد مشارکت‌کننده، نمره تمرین صفر و به استاد گزارش می‌گردد.
 </li>
 <li>برای مطالعه بیشتر درباره‌ی فرمت مارک‌دون می‌توانید از <a href="https://github.com/tajaddini/Persian-Markdown/blob/master/learn-MD.md">این لینک</a> مطالعه کنید.
 </li>

 </ul>
 </div>